In [1]:
import pandas as pd
import requests

from job_apps.constants import API_HEADERS, URL_DATABASE
from job_apps.api_requests import fetch_database_jsons
from job_apps.data_preprocessing import map_dict, build_dataframe

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
search_response = requests.post(url=URL_DATABASE, headers=API_HEADERS)
search_response_dict = search_response.json()

In [9]:
results = fetch_database_jsons(url=URL_DATABASE, headers=API_HEADERS)

In [36]:
df = build_dataframe(results)
df['date_applied'] = pd.to_datetime(df['date_applied'])
df['month_applied'] = df['date_applied'].map(lambda x: x.date().month)

In [44]:
df.groupby('month_applied').count()

,job_title,company,date_applied,origin,stage,job_description,cover_letter
month_applied,,,,,,,
1,85,85,85,85,85,84,2
2,47,47,47,46,47,46,6
3,2,2,2,2,2,2,2
4,37,37,37,37,37,37,4
5,33,33,33,33,33,33,10
6,18,18,18,18,18,18,17
7,63,63,63,63,63,63,54
9,20,20,20,20,20,20,17


In [42]:
l

['job_title',
 'hui',
 'company',
 'date_applied',
 'origin',
 'stage',
 'job_description',
 'cover_letter',
 'month_applied']

In [33]:
timestamp.date().month

9